In [14]:
import numpy as np
from gensim.models import Word2Vec
from itertools import chain
from collections import Counter
import os
from tqdm import trange

In [2]:
with open('quora_duplicate_questions.tsv', 'r') as f:
    lines = f.readlines()
    lines = [l.split('\t')[3:] for l in lines[1:]]
l = len(lines)

def clean(sent):
    sent = str(sent).lower().split()
    return sent
data = []
for i in range(l):
    try:
        data.append([clean(lines[i][0]), clean(lines[i][1]), lines[i][2]])
    except Exception as e:
        continue
l = len(data)

In [3]:
all_qs = np.concatenate((np.array(data)[:,0], np.array(data)[:,1]), axis=0)
vocab = Counter(chain(*all_qs))
vocab = {k:v for k, v in vocab.items() if v >= 2}
for i in range(l):
    data[i][0] = [x if x in vocab else 'unk' for x in data[i][0]]
    data[i][1] = [x if x in vocab else 'unk' for x in data[i][1]]

In [4]:
m = Word2Vec.load('embeddings.model')
max_length = max([len(i) for i in all_qs])
print(max_length)
batch_size = 5
dim = 200

237


In [5]:
i2w = dict(enumerate(list(vocab.keys()) + ['unk']))
w2i = {v:k for k, v in i2w.items()}
vocab_size = len(i2w)
for i in range(l):
    data[i][0] = [w2i[j] for j in data[i][0]] + [vocab_size] * (max_length -len(data[i][0]))
    data[i][1] = [w2i[j] for j in data[i][1]] + [vocab_size] * (max_length -len(data[i][1]))

In [6]:
def train_test_split(a, b, c, ratio = 0.8):
    mask = np.random.rand(l) < ratio
    return a[mask], b[mask], c[mask], a[~mask], b[~mask], c[~mask]

In [7]:
q1 = np.array([i[0] for i in data])
q2 = np.array([i[1] for i in data])
y = np.array([i[2] for i in data])
print(q1.shape, q2.shape, y.shape)
splitted_data = train_test_split(q1, q2, y)

(404340, 237) (404340, 237) (404340,)


In [8]:
vec = np.zeros([vocab_size + 1, dim])
for i, j in i2w.items():
    vec[i] = m[j]
vec[vocab_size] = [0] * 200

In [10]:
def batch(x, y, z, size):
    idx = np.random.choice(range(len(x)), size=size, replace=False)
    return x[idx], y[idx], z[idx]

In [16]:
if not os.path.exists('data'):
    os.mkdir('data')
for i in trange(1000):
    a, b, c = batch(q1, q2, y, 10)
    np.savez('data/'+str(i + 1000)+'.npz', ques1=a, ques2=b, label=c)

100%|██████████| 1000/1000 [01:15<00:00, 13.20it/s]


In [17]:
np.savez('data/embed.npz', embed=vec)